## Data Loading

In [3624]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import math
import numpy as np

# Import libraries for running ANOVA and post-hoc test
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [3625]:
#from an excel file --- need to install openpyxl dependency

#from github
username = "datagus"
repository = "ASDA2025"
directory = "datasets/homework_week5/world_bank_development_indicators.csv"
github_url = f"https://raw.githubusercontent.com/{username}/{repository}/main/{directory}"
indicators_df = pd.read_csv(github_url)

In [3626]:
#from github
username = "datagus"
repository = "ASDA2025"
directory = "datasets/homework_week5/income.xlsx"
github_url = f"https://raw.githubusercontent.com/{username}/{repository}/main/{directory}"
income_xls = pd.ExcelFile(github_url)

In [3627]:
# Check column names of the indicators dataframe
indicators_df.columns

Index(['country', 'date', 'agricultural_land%', 'forest_land%', 'land_area',
       'avg_precipitation', 'trade_in_services%',
       'control_of_corruption_estimate', 'control_of_corruption_std',
       'access_to_electricity%', 'renewvable_energy_consumption%',
       'electric_power_consumption', 'CO2_emisions',
       'other_greenhouse_emisions', 'population_density', 'inflation_annual%',
       'real_interest_rate', 'risk_premium_on_lending',
       'research_and_development_expenditure%', 'central_goverment_debt%',
       'tax_revenue%', 'expense%', 'goverment_effectiveness_estimate',
       'goverment_effectiveness_std', 'human_capital_index', 'doing_business',
       'time_to_get_operation_license', 'statistical_performance_indicators',
       'individuals_using_internet%', 'logistic_performance_index',
       'military_expenditure%', 'GDP_current_US',
       'political_stability_estimate', 'political_stability_std',
       'rule_of_law_estimate', 'rule_of_law_std',
       'reg

In [3628]:
# Preview the indicators dataframe
indicators_df.head()

,country,date,agricultural_land%,forest_land%,land_area,avg_precipitation,trade_in_services%,control_of_corruption_estimate,control_of_corruption_std,access_to_electricity%,...,multidimensional_poverty_headcount_ratio%,gini_index,birth_rate,death_rate,life_expectancy_at_birth,population,rural_population,voice_and_accountability_estimate,voice_and_accountability_std,intentional_homicides
0,Afghanistan,1960-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,50.34,31.92,32.53,8622466.00,7898093.00,NaN,NaN,NaN
1,Afghanistan,1961-01-01,57.88,NaN,652230.00,327.00,NaN,NaN,NaN,NaN,...,NaN,NaN,50.44,31.35,33.07,8790140.00,8026804.00,NaN,NaN,NaN
2,Afghanistan,1962-01-01,57.96,NaN,652230.00,327.00,NaN,NaN,NaN,NaN,...,NaN,NaN,50.57,30.84,33.55,8969047.00,8163985.00,NaN,NaN,NaN
3,Afghanistan,1963-01-01,58.03,NaN,652230.00,327.00,NaN,NaN,NaN,NaN,...,NaN,NaN,50.70,30.36,34.02,9157465.00,8308019.00,NaN,NaN,NaN
4,Afghanistan,1964-01-01,58.12,NaN,652230.00,327.00,NaN,NaN,NaN,NaN,...,NaN,NaN,50.83,29.87,34.49,9355514.00,8458694.00,NaN,NaN,NaN


In [3629]:
# Get a concise summary of the dataframe
indicators_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17272 entries, 0 to 17271
Data columns (total 50 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   country                                    17272 non-null  object 
 1   date                                       17272 non-null  object 
 2   agricultural_land%                         14714 non-null  float64
 3   forest_land%                               8176 non-null   float64
 4   land_area                                  14930 non-null  float64
 5   avg_precipitation                          10086 non-null  float64
 6   trade_in_services%                         9195 non-null   float64
 7   control_of_corruption_estimate             4783 non-null   float64
 8   control_of_corruption_std                  4783 non-null   float64
 9   access_to_electricity%                     7348 non-null   float64
 10  renewvable_energy_cons

## Data Cleansing and Merging

In [3630]:
# Drop redundant columns, columns showing statistical output (not indicator itself), columns with high sparsity and technically complex or irrelevant indicators
columns_to_drop = [
# 1.Redundant and statistical columns (8 Columns)    
    'control_of_corruption_std',
    'goverment_effectiveness_std',
    'political_stability_std',
    'rule_of_law_std',
    'regulatory_quality_std',
    'voice_and_accountability_std',
    'other_greenhouse_emisions',  
    'land_area',                 

# 2. Columns with high missing values (10 Columns)
    'human_capital_index',
    'doing_business',
    'time_to_get_operation_license',
    'multidimensional_poverty_headcount_ratio%',
    'statistical_performance_indicators',
    'logistic_performance_index',
    'central_goverment_debt%',
    'gini_index',
    'risk_premium_on_lending',
    'research_and_development_expenditure%',

# 3. Irrelevent Indicators (12 Columns)
    'avg_precipitation',          
    'trade_in_services%',         
    'real_interest_rate',         
    'expense%',                   
    'electric_power_consumption', 
    'individuals_using_internet%',
    'military_expenditure%', 
    'regulatory_quality_estimate', 
    'birth_rate',                 
    'death_rate',                 
    'rural_population',    
    'population',       
    'intentional_homicides'       
]

# Create the clean DataFrame by dropping the columns
cleanindicators_df = indicators_df.drop(columns=columns_to_drop)

# Get a concise summary of the cleaned dataframe
cleanindicators_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17272 entries, 0 to 17271
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   country                               17272 non-null  object 
 1   date                                  17272 non-null  object 
 2   agricultural_land%                    14714 non-null  float64
 3   forest_land%                          8176 non-null   float64
 4   control_of_corruption_estimate        4783 non-null   float64
 5   access_to_electricity%                7348 non-null   float64
 6   renewvable_energy_consumption%        8076 non-null   float64
 7   CO2_emisions                          7408 non-null   float64
 8   population_density                    14901 non-null  float64
 9   inflation_annual%                     10788 non-null  float64
 10  tax_revenue%                          5125 non-null   float64
 11  goverment_effec

In [3631]:
# Preview the cleaned indicators dataframe
cleanindicators_df.head()

,country,date,agricultural_land%,forest_land%,control_of_corruption_estimate,access_to_electricity%,renewvable_energy_consumption%,CO2_emisions,population_density,inflation_annual%,tax_revenue%,goverment_effectiveness_estimate,GDP_current_US,political_stability_estimate,rule_of_law_estimate,government_expenditure_on_education%,government_health_expenditure%,life_expectancy_at_birth,voice_and_accountability_estimate
0,Afghanistan,1960-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,537777811.11,NaN,NaN,NaN,NaN,32.53,NaN
1,Afghanistan,1961-01-01,57.88,NaN,NaN,NaN,NaN,NaN,13.48,NaN,NaN,NaN,548888895.56,NaN,NaN,NaN,NaN,33.07,NaN
2,Afghanistan,1962-01-01,57.96,NaN,NaN,NaN,NaN,NaN,13.75,NaN,NaN,NaN,546666677.78,NaN,NaN,NaN,NaN,33.55,NaN
3,Afghanistan,1963-01-01,58.03,NaN,NaN,NaN,NaN,NaN,14.04,NaN,NaN,NaN,751111191.11,NaN,NaN,NaN,NaN,34.02,NaN
4,Afghanistan,1964-01-01,58.12,NaN,NaN,NaN,NaN,NaN,14.34,NaN,NaN,NaN,800000044.44,NaN,NaN,NaN,NaN,34.49,NaN


In [3632]:
# List all sheets of income excel file
print(income_xls.sheet_names)

['List of economies', 'composition', 'notes']


In [3633]:
# Parse and create the different dataframes:
economies_df = income_xls.parse('List of economies')
#composition_df = income_xls.parse('composition')

In [3634]:
# Preview the economies dataframe
economies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Economy           266 non-null    object
 1   Code              266 non-null    object
 2   Region            218 non-null    object
 3   Income group      216 non-null    object
 4   Lending category  145 non-null    object
dtypes: object(5)
memory usage: 10.6+ KB


In [3635]:
# Drop rows with missing 'Region' values
economies_df.dropna(subset=['Region'], inplace=True)
economies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 218 entries, 0 to 217
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Economy           218 non-null    object
 1   Code              218 non-null    object
 2   Region            218 non-null    object
 3   Income group      216 non-null    object
 4   Lending category  145 non-null    object
dtypes: object(5)
memory usage: 10.2+ KB


In [3636]:
# Preview the economies dataframe
economies_df.head()

,Economy,Code,Region,Income group,Lending category
0,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA
1,Albania,ALB,Europe & Central Asia,Upper middle income,IBRD
2,Algeria,DZA,"Middle East, North Africa, Afghanistan & Pakistan",Upper middle income,IBRD
3,American Samoa,ASM,East Asia & Pacific,High income,NaN
4,Andorra,AND,Europe & Central Asia,High income,NaN


In [3637]:
# Unique country names from economies_df
unique_economies = economies_df['Economy'].unique()

# Unique country names from cleanindicators_df
unique_indicators = cleanindicators_df['country'].unique()

print("Unique countries in economies_df:", len(unique_economies))
print(unique_economies[:20])  # show first 20 for preview

print("\nUnique countries in cleanindicators_df:", len(unique_indicators))
print(unique_indicators[:20])

Unique countries in economies_df: 218
['Afghanistan' 'Albania' 'Algeria' 'American Samoa' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Aruba' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas, The' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize']

Unique countries in cleanindicators_df: 274
['Afghanistan' 'Africa Eastern and Southern' 'Africa Western and Central'
 'Albania' 'Algeria' 'American Samoa' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Arab World' 'Argentina' 'Armenia' 'Aruba'
 'Australia' 'Austria' 'Azerbaijan' 'Bahamas, The' 'Bahrain' 'Bangladesh'
 'Barbados']


In [3638]:
# Convert to sets
set_econ = set(economies_df['Economy'])
set_ind = set(cleanindicators_df['country'])

# Find differences
only_in_economies = set_econ - set_ind
only_in_indicators = set_ind - set_econ

print("Countries only in economies_df (not in cleanindicators_df):", len(only_in_economies))
print(sorted(list(only_in_economies))[:20])  # show first 20

print("\nCountries only in cleanindicators_df (not in economies_df):", len(only_in_indicators))
print(sorted(list(only_in_indicators)))


Countries only in economies_df (not in cleanindicators_df): 7
['Curaçao', 'Côte d’Ivoire', 'Puerto Rico (U.S.)', 'Somalia, Fed. Rep.', 'São Tomé and Príncipe', 'Taiwan, China', 'Türkiye']

Countries only in cleanindicators_df (not in economies_df): 63
['Africa Eastern and Southern', 'Africa Western and Central', 'Arab World', 'Caribbean small states', 'Central Europe and the Baltics', "Cote d'Ivoire", 'Curacao', 'Czech Republic', 'Early-demographic dividend', 'East Asia & Pacific', 'East Asia & Pacific (IDA & IBRD countries)', 'East Asia & Pacific (IDA & IBRD)', 'East Asia & Pacific (excluding high income)', 'Euro area', 'Europe & Central Asia', 'Europe & Central Asia (IDA & IBRD countries)', 'Europe & Central Asia (IDA & IBRD)', 'Europe & Central Asia (excluding high income)', 'European Union', 'Fragile and conflict affected situations', 'Heavily indebted poor countries (HIPC)', 'High income', 'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only', 'IDA total', 'Late-demographic div

In [3639]:
# Remove aggregate groups from cleanindicators_df
groups_to_remove = [
    'Africa Eastern and Southern', 'Africa Western and Central', 'Arab World',
    'Caribbean small states', 'Central Europe and the Baltics', 'Early-demographic dividend',
    'East Asia & Pacific', 'East Asia & Pacific (IDA & IBRD countries)',
    'East Asia & Pacific (IDA & IBRD)', 'East Asia & Pacific (excluding high income)',
    'Euro area', 'Europe & Central Asia', 'Europe & Central Asia (IDA & IBRD countries)',
    'Europe & Central Asia (IDA & IBRD)', 'Europe & Central Asia (excluding high income)',
    'European Union', 'Fragile and conflict affected situations',
    'Heavily indebted poor countries (HIPC)', 'High income', 'IBRD only',
    'IDA & IBRD total', 'IDA blend', 'IDA only', 'IDA total',
    'Late-demographic dividend', 'Latin America & Caribbean',
    'Latin America & Caribbean (excluding high income)',
    'Latin America & Caribbean (IDA & IBRD)', 'Least developed countries: UN classification',
    'Low & middle income', 'Low income', 'Lower middle income',
    'Middle East, North Africa, Afghanistan and Pakistan',
    'Middle East, North Africa, Afghanistan and Pakistan (excluding high income)',
    'Middle East, North Africa, Afghanistan and Pakistan (IDA & IBRD)',
    'Middle income', 'North America', 'OECD members', 'Other small states',
    'Pacific island small states', 'Post-demographic dividend', 'Pre-demographic dividend',
    'Small states', 'South Asia', 'South Asia (IDA & IBRD)',
    'Sub-Saharan Africa', 'Sub-Saharan Africa (excluding high income)',
    'Sub-Saharan Africa (IDA & IBRD)', 'Upper middle income', 'World', 'Middle East & North Africa', 'Middle East & North Africa (IDA & IBRD countries)',
    'Middle East & North Africa (IDA & IBRD)',
    'Middle East & North Africa (excluding high income)',
    'Not classified','Sub-Saharan Africa (IDA & IBRD countries)','Latin America & the Caribbean (IDA & IBRD countries)'
]

# Filter out the groups from cleanindicators_df
cleanindicators_df = cleanindicators_df[~cleanindicators_df['country'].isin(groups_to_remove)]

# Print remaining unique countries
print("Remaining unique countries:", cleanindicators_df['country'].nunique())

# Convert to sets
set_econ = set(economies_df['Economy'])
set_ind = set(cleanindicators_df['country'])

# Find differences
only_in_economies = set_econ - set_ind
only_in_indicators = set_ind - set_econ

print("\nCountries only in economies_df (not in cleanindicators_df):", len(only_in_economies))
print(sorted(list(only_in_economies))[:20])  # show first 20

print("\nCountries only in cleanindicators_df (not in economies_df):", len(only_in_indicators))
print(sorted(list(only_in_indicators)))


Remaining unique countries: 220

Countries only in economies_df (not in cleanindicators_df): 7
['Curaçao', 'Côte d’Ivoire', 'Puerto Rico (U.S.)', 'Somalia, Fed. Rep.', 'São Tomé and Príncipe', 'Taiwan, China', 'Türkiye']

Countries only in cleanindicators_df (not in economies_df): 9
["Cote d'Ivoire", 'Curacao', 'Czech Republic', 'Puerto Rico', 'Sao Tome and Principe', 'Somalia', 'Turkey', 'Turkiye', 'Viet Nam']


In [3640]:
# Mapping accented names to simpler names
economies_name_mapping = {
    "Curaçao": "Curacao",
    "Côte d’Ivoire": "Cote d'Ivoire",
    "Puerto Rico (U.S.)": "Puerto Rico",
    "Somalia, Fed. Rep.": "Somalia",
    "São Tomé and Príncipe": "Sao Tome and Principe",
    "Türkiye": "Turkey",
    "Czechia": "Czech Republic"  # optional depending on your cleanindicators_df
}

economies_df['Economy'] = economies_df['Economy'].replace(economies_name_mapping)

# Drop the entry
economies_df = economies_df[economies_df['Economy'] != "Taiwan, China"]

# Convert to sets
set_econ = set(economies_df['Economy'])
set_ind = set(cleanindicators_df['country'])

# Find differences
only_in_economies = set_econ - set_ind
only_in_indicators = set_ind - set_econ

print("Countries only in economies_df (not in cleanindicators_df):", len(only_in_economies))
print(sorted(list(only_in_economies))[:20])  # show first 20

print("\nCountries only in cleanindicators_df (not in economies_df):", len(only_in_indicators))
print(sorted(list(only_in_indicators)))



Countries only in economies_df (not in cleanindicators_df): 0
[]

Countries only in cleanindicators_df (not in economies_df): 3
['Czechia', 'Turkiye', 'Viet Nam']


In [3641]:
# Check data in specific country names
variant_names = ['Czechia', 'Turkiye', 'Viet Nam']

# Show non-NA counts per column for these rows
print(cleanindicators_df[cleanindicators_df['country'].isin(variant_names)].info())



<class 'pandas.core.frame.DataFrame'>
Index: 192 entries, 3679 to 16856
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   country                               192 non-null    object 
 1   date                                  192 non-null    object 
 2   agricultural_land%                    151 non-null    float64
 3   forest_land%                          93 non-null     float64
 4   control_of_corruption_estimate        72 non-null     float64
 5   access_to_electricity%                79 non-null     float64
 6   renewvable_energy_consumption%        93 non-null     float64
 7   CO2_emisions                          93 non-null     float64
 8   population_density                    151 non-null    float64
 9   inflation_annual%                     121 non-null    float64
 10  tax_revenue%                          70 non-null     float64
 11  goverment_effective

In [3642]:
# Check data in specific country names 
variant_names = ['Czech Republic', 'Turkey', 'Vietnam']

# Show non-NA counts per column for these rows
print(cleanindicators_df[cleanindicators_df['country'].isin(variant_names)].info())



<class 'pandas.core.frame.DataFrame'>
Index: 93 entries, 3648 to 16887
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   country                               93 non-null     object 
 1   date                                  93 non-null     object 
 2   agricultural_land%                    0 non-null      float64
 3   forest_land%                          0 non-null      float64
 4   control_of_corruption_estimate        0 non-null      float64
 5   access_to_electricity%                0 non-null      float64
 6   renewvable_energy_consumption%        0 non-null      float64
 7   CO2_emisions                          0 non-null      float64
 8   population_density                    0 non-null      float64
 9   inflation_annual%                     0 non-null      float64
 10  tax_revenue%                          0 non-null      float64
 11  goverment_effectiven

In [3643]:
# List of countries to remove
countries_to_remove = ['Czech Republic', 'Turkey', 'Vietnam']

# Keep only rows where country is NOT in the list
cleanindicators_df = cleanindicators_df[~cleanindicators_df['country'].isin(countries_to_remove)]

# Define the mapping
name_mapping = {
    'Czechia': 'Czech Republic',
    'Turkiye': 'Turkey',
    'Viet Nam': 'Vietnam'
}

# Apply the mapping
cleanindicators_df['country'] = cleanindicators_df['country'].replace(name_mapping)

# Convert to sets
set_econ = set(economies_df['Economy'])
set_ind = set(cleanindicators_df['country'])

# Find differences
only_in_economies = set_econ - set_ind
only_in_indicators = set_ind - set_econ

print("Countries only in economies_df (not in cleanindicators_df):", len(only_in_economies))
print(sorted(list(only_in_economies))[:20])  # show first 20

print("\nCountries only in cleanindicators_df (not in economies_df):", len(only_in_indicators))
print(sorted(list(only_in_indicators)))


Countries only in economies_df (not in cleanindicators_df): 0
[]

Countries only in cleanindicators_df (not in economies_df): 0
[]


In [3644]:
cols_to_check = [
    'agricultural_land%', 'forest_land%', 'control_of_corruption_estimate', 'access_to_electricity%',
    'renewvable_energy_consumption%', 'CO2_emisions', 'population_density', 'inflation_annual%',
    'tax_revenue%', 'goverment_effectiveness_estimate', 'GDP_current_US', 'political_stability_estimate',
    'rule_of_law_estimate', 'government_expenditure_on_education%', 'government_health_expenditure%',
    'life_expectancy_at_birth', 'voice_and_accountability_estimate'
]

total_rows = len(cleanindicators_df)

for col in cols_to_check:
    non_numeric_count = pd.to_numeric(cleanindicators_df[col], errors='coerce').isna().sum()
    non_numeric_percent = (non_numeric_count / total_rows) * 100
    print(f"{col}: {non_numeric_percent:.2f}% non-numeric values")


agricultural_land%: 15.14% non-numeric values
forest_land%: 52.04% non-numeric values
control_of_corruption_estimate: 65.56% non-numeric values
access_to_electricity%: 56.06% non-numeric values
renewvable_energy_consumption%: 52.56% non-numeric values
CO2_emisions: 57.37% non-numeric values
population_density: 13.79% non-numeric values
inflation_annual%: 38.31% non-numeric values
tax_revenue%: 69.10% non-numeric values
goverment_effectiveness_estimate: 65.73% non-numeric values
GDP_current_US: 24.62% non-numeric values
political_stability_estimate: 65.29% non-numeric values
rule_of_law_estimate: 64.91% non-numeric values
government_expenditure_on_education%: 64.54% non-numeric values
government_health_expenditure%: 71.70% non-numeric values
life_expectancy_at_birth: 7.19% non-numeric values
voice_and_accountability_estimate: 65.08% non-numeric values


In [3645]:
# Columns to bring in from economies_df
columns_to_add = ['Economy', 'Code', 'Region', 'Income group', 'Lending category']

# Merge
merged_df = cleanindicators_df.merge(
    economies_df[columns_to_add],
    how='left',
    left_on='country',
    right_on='Economy'
)

# Drop duplicate 'Economy' column
merged_df.drop(columns=['Economy'], inplace=True)

# Move the new columns right after 'country'
for i, col in enumerate(['Code', 'Region', 'Income group', 'Lending category']):
    merged_df.insert(i+1, col, merged_df.pop(col))

# Check
merged_df.head()


,country,Code,Region,Income group,Lending category,date,agricultural_land%,forest_land%,control_of_corruption_estimate,access_to_electricity%,...,inflation_annual%,tax_revenue%,goverment_effectiveness_estimate,GDP_current_US,political_stability_estimate,rule_of_law_estimate,government_expenditure_on_education%,government_health_expenditure%,life_expectancy_at_birth,voice_and_accountability_estimate
0,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1960-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,537777811.11,NaN,NaN,NaN,NaN,32.53,NaN
1,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1961-01-01,57.88,NaN,NaN,NaN,...,NaN,NaN,NaN,548888895.56,NaN,NaN,NaN,NaN,33.07,NaN
2,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1962-01-01,57.96,NaN,NaN,NaN,...,NaN,NaN,NaN,546666677.78,NaN,NaN,NaN,NaN,33.55,NaN
3,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1963-01-01,58.03,NaN,NaN,NaN,...,NaN,NaN,NaN,751111191.11,NaN,NaN,NaN,NaN,34.02,NaN
4,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1964-01-01,58.12,NaN,NaN,NaN,...,NaN,NaN,NaN,800000044.44,NaN,NaN,NaN,NaN,34.49,NaN


In [3646]:
# Check datatypes in merged dataframe
print(merged_df.dtypes)


country                                  object
Code                                     object
Region                                   object
Income group                             object
Lending category                         object
date                                     object
agricultural_land%                      float64
forest_land%                            float64
control_of_corruption_estimate          float64
access_to_electricity%                  float64
renewvable_energy_consumption%          float64
CO2_emisions                            float64
population_density                      float64
inflation_annual%                       float64
tax_revenue%                            float64
goverment_effectiveness_estimate        float64
GDP_current_US                          float64
political_stability_estimate            float64
rule_of_law_estimate                    float64
government_expenditure_on_education%    float64
government_health_expenditure%          

In [3647]:
# Checking missing values before imputation

columns_to_impute = [
    'agricultural_land%',
    'forest_land%',
    'control_of_corruption_estimate',
    'access_to_electricity%',
    'renewvable_energy_consumption%',
    'CO2_emisions',
    'population_density',
    'inflation_annual%',
    'tax_revenue%',
    'goverment_effectiveness_estimate',
    'GDP_current_US',
    'rule_of_law_estimate',
    'government_expenditure_on_education%',
    'government_health_expenditure%',
    'life_expectancy_at_birth',
    'voice_and_accountability_estimate', 'political_stability_estimate'
]

missing_before = merged_df[columns_to_impute].isna().sum()
print("Missing values BEFORE imputation:")
print(missing_before)

Missing values BEFORE imputation:
agricultural_land%                      2102
forest_land%                            7228
control_of_corruption_estimate          9105
access_to_electricity%                  7786
renewvable_energy_consumption%          7300
CO2_emisions                            7968
population_density                      1915
inflation_annual%                       5321
tax_revenue%                            9596
goverment_effectiveness_estimate        9129
GDP_current_US                          3419
rule_of_law_estimate                    9015
government_expenditure_on_education%    8963
government_health_expenditure%          9958
life_expectancy_at_birth                 998
voice_and_accountability_estimate       9038
political_stability_estimate            9068
dtype: int64


In [3648]:
# Imputing missing values using the median value of the same country's indicator

for col in columns_to_impute:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

# Impute missing values per country using median
for col in columns_to_impute:
    merged_df[col] = merged_df[col].fillna(
        merged_df.groupby('country')[col].transform('median')
    )

# Check remaining missing values
missing_after = merged_df[columns_to_impute].isna().sum()
print("Missing values after imputation:")
print(missing_after)

Missing values after imputation:
agricultural_land%                       448
forest_land%                             192
control_of_corruption_estimate           768
access_to_electricity%                    64
renewvable_energy_consumption%           320
CO2_emisions                            1664
population_density                        64
inflation_annual%                       1600
tax_revenue%                            3648
goverment_effectiveness_estimate         768
GDP_current_US                           192
rule_of_law_estimate                     768
government_expenditure_on_education%    1024
government_health_expenditure%          1728
life_expectancy_at_birth                 320
voice_and_accountability_estimate        768
political_stability_estimate             768
dtype: int64


In [3649]:
# check which countries have missing indicators

missing_df = pd.DataFrame(list(country_missing_count.items()), 
                          columns=['country', 'missing_count'])

# Sort by missing_count descending
missing_df = missing_df.sort_values('missing_count', ascending=False)

# Print results
for row in missing_df.itertuples(index=False):
    print(f"{row.country}: {row.missing_count} missing indicators")


Gibraltar: 12 missing indicators
St. Martin (French part): 12 missing indicators
Channel Islands: 11 missing indicators
Northern Mariana Islands: 11 missing indicators
French Polynesia: 10 missing indicators
Sint Maarten (Dutch part): 10 missing indicators
British Virgin Islands: 10 missing indicators
Isle of Man: 10 missing indicators
Faroe Islands: 10 missing indicators
Turks and Caicos Islands: 9 missing indicators
New Caledonia: 9 missing indicators
Curacao: 9 missing indicators
Kosovo: 8 missing indicators
Monaco: 6 missing indicators
American Samoa: 6 missing indicators
Greenland: 5 missing indicators
Korea, Dem. People's Rep.: 5 missing indicators
Guam: 5 missing indicators
Virgin Islands (U.S.): 5 missing indicators
Bermuda: 4 missing indicators
Hong Kong SAR, China: 4 missing indicators
Puerto Rico: 4 missing indicators
Macao SAR, China: 4 missing indicators
Aruba: 3 missing indicators
San Marino: 3 missing indicators
Cayman Islands: 3 missing indicators
Andorra: 3 missing ind

In [3650]:
# List of columns to check for missing values
columns_to_check = [
    'agricultural_land%',
    'forest_land%',
    'control_of_corruption_estimate',
    'access_to_electricity%',
    'renewvable_energy_consumption%',
    'CO2_emisions',
    'population_density',
    'inflation_annual%',
    'tax_revenue%',
    'goverment_effectiveness_estimate',
    'GDP_current_US',
    'rule_of_law_estimate',
    'government_expenditure_on_education%',
    'government_health_expenditure%',
    'life_expectancy_at_birth',
    'voice_and_accountability_estimate', 'political_stability_estimate'
]

# Strip whitespace from country names to avoid mismatches
merged_df['country'] = merged_df['country'].str.strip()

# Count missing indicators per country
country_missing_count = {}
for col in columns_to_check:
    missing_rows = merged_df[merged_df[col].isna()]
    for c in missing_rows['country'].unique():
        country_missing_count[c] = country_missing_count.get(c, 0) + 1

# Identify countries with 8 or more missing indicators
countries_to_remove = [c for c, count in country_missing_count.items() if count >= 8]

print("Countries removed (8+ missing indicators):")
for c in countries_to_remove:
    print(c)

# Remove those countries
cleanmerged_df = merged_df[~merged_df['country'].isin(countries_to_remove)].copy()

# Show summary
print("\nOriginal number of countries:", merged_df['country'].nunique())
print("Remaining after removal:", cleanmerged_df['country'].nunique())


Countries removed (8+ missing indicators):
Curacao
Gibraltar
Kosovo
Sint Maarten (Dutch part)
St. Martin (French part)
British Virgin Islands
Channel Islands
Faroe Islands
French Polynesia
Isle of Man
New Caledonia
Northern Mariana Islands
Turks and Caicos Islands

Original number of countries: 217
Remaining after removal: 204


In [3651]:
# Count missing values per country
missing_counts2 = {}

for col in columns_to_check:
    missing_rows2 = cleanmerged_df[cleanmerged_df[col].isna()]
    countries = missing_rows2["country"].tolist()
    
    for c in countries:
        if c not in missing_counts2:
            missing_counts2[c] = 1
        else:
            missing_counts2[c] += 1

# Convert dictionary to a list of tuples
missing_list2 = list(missing_counts2.items())

# Sort the list by missing count (descending)
for i in range(len(missing_list2)):
    for j in range(i + 1, len(missing_list2)):
        if missing_list2[j][1] > missing_list2[i][1]:
            missing_list2[i], missing_list2[j] = missing_list2[j], missing_list2[i]

# Print only countries with missing values
print("Countries still missing indicators:")
for country, count in missing_list2:
    print(f"{country}: {count} missing values")


Countries still missing indicators:
Monaco: 384 missing values
American Samoa: 384 missing values
Greenland: 320 missing values
Guam: 320 missing values
Virgin Islands (U.S.): 320 missing values
Korea, Dem. People's Rep.: 320 missing values
Macao SAR, China: 256 missing values
Puerto Rico: 256 missing values
Bermuda: 256 missing values
Hong Kong SAR, China: 256 missing values
Cayman Islands: 192 missing values
Aruba: 192 missing values
Andorra: 192 missing values
San Marino: 192 missing values
Liechtenstein: 192 missing values
Eritrea: 128 missing values
West Bank and Gaza: 128 missing values
Cuba: 128 missing values
Somalia: 128 missing values
Turkmenistan: 128 missing values
Montenegro: 128 missing values
Marshall Islands: 64 missing values
Algeria: 64 missing values
Antigua and Barbuda: 64 missing values
Brunei Darussalam: 64 missing values
Chad: 64 missing values
Comoros: 64 missing values
Djibouti: 64 missing values
Dominica: 64 missing values
Eswatini: 64 missing values
Grenada: 

In [3652]:
# Columns to impute
columns_to_impute = [
    'agricultural_land%',
    'forest_land%',
    'control_of_corruption_estimate',
    'access_to_electricity%',
    'renewvable_energy_consumption%',
    'CO2_emisions',
    'population_density',
    'inflation_annual%',
    'tax_revenue%',
    'goverment_effectiveness_estimate',
    'GDP_current_US',
    'rule_of_law_estimate',
    'government_expenditure_on_education%',
    'government_health_expenditure%',
    'life_expectancy_at_birth',
    'voice_and_accountability_estimate',
    'political_stability_estimate'
]

# Ensure numeric
for col in columns_to_impute:
    cleanmerged_df[col] = pd.to_numeric(cleanmerged_df[col], errors='coerce')

# Impute missing values using region + income group median
for col in columns_to_impute:
    cleanmerged_df[col] = cleanmerged_df[col].fillna(
        cleanmerged_df.groupby(['Region', 'Income group'])[col].transform('median')
    )

# Check remaining missing values
missing_after_region_group = cleanmerged_df[columns_to_impute].isna().sum()
print("Missing values after region + income group imputation:")
print(missing_after_region_group)


Missing values after region + income group imputation:
agricultural_land%                        0
forest_land%                              0
control_of_corruption_estimate            0
access_to_electricity%                    0
renewvable_energy_consumption%            0
CO2_emisions                              0
population_density                        0
inflation_annual%                        64
tax_revenue%                            128
goverment_effectiveness_estimate          0
GDP_current_US                           64
rule_of_law_estimate                      0
government_expenditure_on_education%     64
government_health_expenditure%           64
life_expectancy_at_birth                  0
voice_and_accountability_estimate         0
political_stability_estimate              0
dtype: int64


In [3653]:
# Counting missing indicators per country
remaining_missing_counts = {}

# Loop through each column and record missing countries
for col in columns_to_impute:
    missing_rows = cleanmerged_df[cleanmerged_df[col].isna()]
    countries = missing_rows['country'].tolist()
    
    for c in countries:
        if c not in remaining_missing_counts:
            remaining_missing_counts[c] = 1
        else:
            remaining_missing_counts[c] += 1

# Sort countries by number of missing indicators (descending)
remaining_missing_list = list(remaining_missing_counts.items())

# Simple sort
for i in range(len(remaining_missing_list)):
    for j in range(i + 1, len(remaining_missing_list)):
        if remaining_missing_list[j][1] > remaining_missing_list[i][1]:
            remaining_missing_list[i], remaining_missing_list[j] = remaining_missing_list[j], remaining_missing_list[i]

# Display countries still missing indicators
print("Countries still missing indicators after region + income group imputation:")
for country, count in remaining_missing_list:
    print(f"{country}: {count} missing indicators")


Countries still missing indicators after region + income group imputation:
Korea, Dem. People's Rep.: 320 missing indicators
Venezuela, RB: 64 missing indicators


In [3654]:
# Columns to impute
cols = [
    'agricultural_land%', 'forest_land%', 'control_of_corruption_estimate',
    'access_to_electricity%', 'renewvable_energy_consumption%', 'CO2_emisions',
    'population_density', 'inflation_annual%', 'tax_revenue%',
    'goverment_effectiveness_estimate', 'GDP_current_US', 'rule_of_law_estimate',
    'government_expenditure_on_education%', 'government_health_expenditure%',
    'life_expectancy_at_birth', 'voice_and_accountability_estimate', 'political_stability_estimate'
]

# Mask for Korea
mask = cleanmerged_df['country'] == "Korea, Dem. People's Rep."

# Impute each column
for c in cols:
    if pd.isna(cleanmerged_df.loc[mask, c].values[0]):
        candidates = cleanmerged_df[
            (cleanmerged_df['Region'] == cleanmerged_df.loc[mask, 'Region'].values[0]) &
            (cleanmerged_df['Income group'] == 'Lower middle income') &
            (~cleanmerged_df[c].isna())
        ][c]
        if not candidates.empty:
            cleanmerged_df.loc[mask, c] = candidates.median()

# Check missing values for Korea
print(cleanmerged_df.loc[mask, cols].isna().sum())


agricultural_land%                      0
forest_land%                            0
control_of_corruption_estimate          0
access_to_electricity%                  0
renewvable_energy_consumption%          0
CO2_emisions                            0
population_density                      0
inflation_annual%                       0
tax_revenue%                            0
goverment_effectiveness_estimate        0
GDP_current_US                          0
rule_of_law_estimate                    0
government_expenditure_on_education%    0
government_health_expenditure%          0
life_expectancy_at_birth                0
voice_and_accountability_estimate       0
political_stability_estimate            0
dtype: int64


In [3655]:
# Venezuela mask
mask_venezuela = cleanmerged_df['country'] == 'Venezuela, RB'

# Get the region of Venezuela
region_venezuela = cleanmerged_df.loc[mask_venezuela, 'Region'].values[0]

# Most frequent income group in the region
most_common_income = cleanmerged_df[cleanmerged_df['Region'] == region_venezuela]['Income group'] \
    .mode()[0]

# Assign to Venezuela
cleanmerged_df.loc[mask_venezuela, 'Income group'] = most_common_income

# Check
print(cleanmerged_df.loc[mask_venezuela, ['Region', 'Income group']])


                          Region         Income group
13440  Latin America & Caribbean  Upper middle income
13441  Latin America & Caribbean  Upper middle income
13442  Latin America & Caribbean  Upper middle income
13443  Latin America & Caribbean  Upper middle income
13444  Latin America & Caribbean  Upper middle income
13445  Latin America & Caribbean  Upper middle income
13446  Latin America & Caribbean  Upper middle income
13447  Latin America & Caribbean  Upper middle income
13448  Latin America & Caribbean  Upper middle income
13449  Latin America & Caribbean  Upper middle income
13450  Latin America & Caribbean  Upper middle income
13451  Latin America & Caribbean  Upper middle income
13452  Latin America & Caribbean  Upper middle income
13453  Latin America & Caribbean  Upper middle income
13454  Latin America & Caribbean  Upper middle income
13455  Latin America & Caribbean  Upper middle income
13456  Latin America & Caribbean  Upper middle income
13457  Latin America & Carib

In [3656]:
# List of columns still missing for Venezuela
columns_to_impute_venezuela = [
    'inflation_annual%',
    'tax_revenue%',
    'GDP_current_US',
    'government_expenditure_on_education%',
    'government_health_expenditure%'
]

# Loop through each column and impute using median of countries with same Region and Income group
for col in columns_to_impute_venezuela:
    median_value = cleanmerged_df[
        (cleanmerged_df['Region'] == 'Latin America & Caribbean') & 
        (cleanmerged_df['Income group'] == 'Upper middle income')
    ][col].median()
    
    cleanmerged_df.loc[cleanmerged_df['country'] == 'Venezuela, RB', col] = median_value

# Check Venezuela after imputation
venezuela_missing = cleanmerged_df[cleanmerged_df['country'] == 'Venezuela, RB'][columns_to_impute_venezuela].isna().sum()
print("Missing values for Venezuela after region + income group imputation:")
print(venezuela_missing)


Missing values for Venezuela after region + income group imputation:
inflation_annual%                       0
tax_revenue%                            0
GDP_current_US                          0
government_expenditure_on_education%    0
government_health_expenditure%          0
dtype: int64


In [3657]:
# Check missing values in all columns
print(cleanmerged_df.isna().sum())

country                                    0
Code                                       0
Region                                     0
Income group                              64
Lending category                        3840
date                                       0
agricultural_land%                         0
forest_land%                               0
control_of_corruption_estimate             0
access_to_electricity%                     0
renewvable_energy_consumption%             0
CO2_emisions                               0
population_density                         0
inflation_annual%                          0
tax_revenue%                               0
goverment_effectiveness_estimate           0
GDP_current_US                             0
political_stability_estimate               0
rule_of_law_estimate                       0
government_expenditure_on_education%       0
government_health_expenditure%             0
life_expectancy_at_birth                   0
voice_and_

In [3658]:
# Select rows with missing Lending category
missing_lending = cleanmerged_df[cleanmerged_df['Lending category'].isna()]

# Find unique countries with missing Lending category and their income group
missing_income_info = missing_lending[['country', 'Income group']].drop_duplicates()

# Automatically fill Lending category for high-income countries
high_income_mask = (cleanmerged_df['Lending category'].isna()) & (cleanmerged_df['Income group'] == "High income")
cleanmerged_df.loc[high_income_mask, 'Lending category'] = "Not classified"

# Check missing values in all columns
print(cleanmerged_df.isna().sum())



country                                   0
Code                                      0
Region                                    0
Income group                             64
Lending category                        192
date                                      0
agricultural_land%                        0
forest_land%                              0
control_of_corruption_estimate            0
access_to_electricity%                    0
renewvable_energy_consumption%            0
CO2_emisions                              0
population_density                        0
inflation_annual%                         0
tax_revenue%                              0
goverment_effectiveness_estimate          0
GDP_current_US                            0
political_stability_estimate              0
rule_of_law_estimate                      0
government_expenditure_on_education%      0
government_health_expenditure%            0
life_expectancy_at_birth                  0
voice_and_accountability_estimat

In [3659]:
# For each row with missing Lending category
for idx, row in cleanmerged_df[cleanmerged_df['Lending category'].isna()].iterrows():
    region = row['Region']
    income = row['Income group']
    
    # Find the most frequent Lending category for the same region and income group
    most_common = cleanmerged_df[
        (cleanmerged_df['Region'] == region) & 
        (cleanmerged_df['Income group'] == income) & 
        (cleanmerged_df['Lending category'].notna())
    ]['Lending category'].mode()
    
    # If there is a most frequent value, assign it
    if not most_common.empty:
        cleanmerged_df.at[idx, 'Lending category'] = most_common[0]

# Check the result
print("Missing values after region + income-level based imputation:")
print(cleanmerged_df.isna().sum())


Missing values after region + income-level based imputation:
country                                  0
Code                                     0
Region                                   0
Income group                            64
Lending category                        64
date                                     0
agricultural_land%                       0
forest_land%                             0
control_of_corruption_estimate           0
access_to_electricity%                   0
renewvable_energy_consumption%           0
CO2_emisions                             0
population_density                       0
inflation_annual%                        0
tax_revenue%                             0
goverment_effectiveness_estimate         0
GDP_current_US                           0
political_stability_estimate             0
rule_of_law_estimate                     0
government_expenditure_on_education%     0
government_health_expenditure%           0
life_expectancy_at_birth            

In [3660]:
# Filter only Korea, Dem. People's Rep.
korea_missing = cleanmerged_df[cleanmerged_df['country'] == "Korea, Dem. People's Rep."]

# Find the most frequent Lending category in the same region and Lower middle income group
region = korea_missing['Region'].iloc[0]
income_level = "Lower middle income"

# Compute mode of Lending category for countries in same region and income level
mode_lending = cleanmerged_df[
    (cleanmerged_df['Region'] == region) &
    (cleanmerged_df['Income group'] == income_level) &
    (cleanmerged_df['Lending category'].notna())
]['Lending category'].mode()

# If mode exists, fill Korea's missing Lending category
if not mode_lending.empty:
    cleanmerged_df.loc[
        cleanmerged_df['country'] == "Korea, Dem. People's Rep.",
        'Lending category'
    ] = mode_lending[0]

# Check the result
print("Missing values after filling Korea's lending category")
print(cleanmerged_df.isna().sum())

Missing values after filling Korea's lending category
country                                  0
Code                                     0
Region                                   0
Income group                            64
Lending category                         0
date                                     0
agricultural_land%                       0
forest_land%                             0
control_of_corruption_estimate           0
access_to_electricity%                   0
renewvable_energy_consumption%           0
CO2_emisions                             0
population_density                       0
inflation_annual%                        0
tax_revenue%                             0
goverment_effectiveness_estimate         0
GDP_current_US                           0
political_stability_estimate             0
rule_of_law_estimate                     0
government_expenditure_on_education%     0
government_health_expenditure%           0
life_expectancy_at_birth                 0


In [3661]:
# Get unique countries with missing Income group
missing_income_countries = cleanmerged_df[cleanmerged_df['Income group'].isna()]['country'].unique()

# Display
print("Countries missing Income group:")
for c in missing_income_countries:
    print(c)


Countries missing Income group:
Ethiopia


In [3662]:
# Impute missing Income group based on region mode
for region in cleanmerged_df['Region'].unique():
    # Get mode of Income group for the region
    region_mode = cleanmerged_df.loc[
        (cleanmerged_df['Region'] == region) & (cleanmerged_df['Income group'].notna()),
        'Income group'
    ].mode()
    
    if not region_mode.empty:
        # Fill missing Income group for countries in this region
        cleanmerged_df.loc[
            (cleanmerged_df['Region'] == region) & (cleanmerged_df['Income group'].isna()),
            'Income group'
        ] = region_mode[0]

# Check the result
print("Missing values after filling Ethopia's income level")
print(cleanmerged_df.isna().sum())


Missing values after filling Ethopia's income level
country                                 0
Code                                    0
Region                                  0
Income group                            0
Lending category                        0
date                                    0
agricultural_land%                      0
forest_land%                            0
control_of_corruption_estimate          0
access_to_electricity%                  0
renewvable_energy_consumption%          0
CO2_emisions                            0
population_density                      0
inflation_annual%                       0
tax_revenue%                            0
goverment_effectiveness_estimate        0
GDP_current_US                          0
political_stability_estimate            0
rule_of_law_estimate                    0
government_expenditure_on_education%    0
government_health_expenditure%          0
life_expectancy_at_birth                0
voice_and_accountability

In [3663]:
# Double check missing values in the final dataframe
cleanmerged_df.isna().sum().sum()


0

In [3664]:
# Define columns where negative or zero values are impossible
impossible_columns = [
    'population_density',
    'GDP_current_US',
    'CO2_emisions',
    'inflation_annual%',
    'tax_revenue%',
    'government_expenditure_on_education%',
    'government_health_expenditure%',
    'life_expectancy_at_birth',
    'agricultural_land%',
    'forest_land%',
    'access_to_electricity%',
    'renewvable_energy_consumption%'
]

# Check for negative or zero values
for col in impossible_columns:
    invalid_rows = cleanmerged_df[cleanmerged_df[col] < 0]
    if not invalid_rows.empty:
        print(f"Column '{col}' has invalid values in countries:")
        print(invalid_rows[['country', col]])
    else:
        print(f"Column '{col}' has no negative values.")


Column 'population_density' has no negative values.
Column 'GDP_current_US' has no negative values.
Column 'CO2_emisions' has no negative values.
Column 'inflation_annual%' has invalid values in countries:
                              country  inflation_annual%
49                        Afghanistan              -6.81
55                        Afghanistan              -0.66
433               Antigua and Barbuda              -0.55
440               Antigua and Barbuda              -0.49
552                           Armenia              -0.79
568                           Armenia              -1.40
625                             Aruba              -2.14
629                             Aruba              -2.37
632                             Aruba              -0.93
633                             Aruba              -1.03
642                         Australia              -0.32
800                        Azerbaijan             -10.63
806                        Azerbaijan              -0

In [3665]:
cleanmerged_df.columns

Index(['country', 'Code', 'Region', 'Income group', 'Lending category', 'date',
       'agricultural_land%', 'forest_land%', 'control_of_corruption_estimate',
       'access_to_electricity%', 'renewvable_energy_consumption%',
       'CO2_emisions', 'population_density', 'inflation_annual%',
       'tax_revenue%', 'goverment_effectiveness_estimate', 'GDP_current_US',
       'political_stability_estimate', 'rule_of_law_estimate',
       'government_expenditure_on_education%',
       'government_health_expenditure%', 'life_expectancy_at_birth',
       'voice_and_accountability_estimate'],
      dtype='object')

In [3666]:
# Step 1: lower-case all columns first
cleanmerged_df.columns = [c.lower() for c in cleanmerged_df.columns]

# Step 2: dictionary for specific renaming / typo fixes
rename_dict = {
    'renewvable_energy_consumption%': 'renewable_energy_consumption_pct',
    'co2_emisions': 'co2_emissions',
    'goverment_effectiveness_estimate': 'gov_effectiveness_estimate',
    'inflation_annual%': 'inflation_yr_pct',
    'government_expenditure_on_education%': 'gov_exp_on_education_pct',
    'government_health_expenditure%': 'gov_health_exp_pct',
    'population_density': 'pop_density'
}

# Apply renaming
cleanmerged_df.rename(columns=rename_dict, inplace=True)

# Step 3: replace spaces, %, and standardize
cols = cleanmerged_df.columns
cols = [c.replace(' ', '_').replace('%', '_pct') for c in cols]
cols = [c.replace('annual', 'yr') for c in cols]
cols = [c.replace('population', 'pop') for c in cols]

# Step 4: replace 'government' with 'gov'
cols = [c.replace('government', 'gov') for c in cols]

# Apply final column names
cleanmerged_df.columns = cols

# Check
cleanmerged_df.columns


Index(['country', 'code', 'region', 'income_group', 'lending_category', 'date',
       'agricultural_land_pct', 'forest_land_pct',
       'control_of_corruption_estimate', 'access_to_electricity_pct',
       'renewable_energy_consumption_pct', 'co2_emissions', 'pop_density',
       'inflation_yr_pct', 'tax_revenue_pct', 'gov_effectiveness_estimate',
       'gdp_current_us', 'political_stability_estimate',
       'rule_of_law_estimate', 'gov_exp_on_education_pct',
       'gov_health_exp_pct', 'life_expectancy_at_birth',
       'voice_and_accountability_estimate'],
      dtype='object')

In [3667]:
# Round all float columns to 2 decimal places first
float_cols = cleanmerged_df.select_dtypes(include='float64').columns
cleanmerged_df[float_cols] = cleanmerged_df[float_cols].round(2)

# Find the index of the old GDP column
gdp_index = cleanmerged_df.columns.get_loc('gdp_current_us')

# Convert GDP to millions and round to 2 decimals
cleanmerged_df['gdp_current_us'] = (cleanmerged_df['gdp_current_us'] / 1_000_000).round(2)

# Move GDP column back to its original position
cols = list(cleanmerged_df.columns)
cols.insert(gdp_index, cols.pop(cols.index('gdp_current_us')))
cleanmerged_df = cleanmerged_df[cols]

# Rename GDP column to indicate millions
cleanmerged_df.rename(columns={'gdp_current_us': 'gdp_current_usd_M'}, inplace=True)

# Optional: Prevent scientific notation for display
pd.set_option('display.float_format', '{:.2f}'.format)

# Check the result
cleanmerged_df.head()


,country,code,region,income_group,lending_category,date,agricultural_land_pct,forest_land_pct,control_of_corruption_estimate,access_to_electricity_pct,...,inflation_yr_pct,tax_revenue_pct,gov_effectiveness_estimate,gdp_current_usd_M,political_stability_estimate,rule_of_law_estimate,gov_exp_on_education_pct,gov_health_exp_pct,life_expectancy_at_birth,voice_and_accountability_estimate
0,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1960-01-01,58.19,1.85,-1.44,45.75,...,4.98,7.65,-1.48,537.78,-2.51,-1.71,3.36,0.53,32.53,-1.19
1,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1961-01-01,57.88,1.85,-1.44,45.75,...,4.98,7.65,-1.48,548.89,-2.51,-1.71,3.36,0.53,33.07,-1.19
2,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1962-01-01,57.96,1.85,-1.44,45.75,...,4.98,7.65,-1.48,546.67,-2.51,-1.71,3.36,0.53,33.55,-1.19
3,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1963-01-01,58.03,1.85,-1.44,45.75,...,4.98,7.65,-1.48,751.11,-2.51,-1.71,3.36,0.53,34.02,-1.19
4,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1964-01-01,58.12,1.85,-1.44,45.75,...,4.98,7.65,-1.48,800.00,-2.51,-1.71,3.36,0.53,34.49,-1.19


In [3668]:
# Replace 'date' column with just the year, in-place
cleanmerged_df['date'] = pd.to_datetime(cleanmerged_df['date'], errors='coerce').dt.year

# Rename 'date' column to 'year'
cleanmerged_df.rename(columns={'date': 'year'}, inplace=True)

# Check
cleanmerged_df.head()

# Check the result
cleanmerged_df.head()


,country,code,region,income_group,lending_category,year,agricultural_land_pct,forest_land_pct,control_of_corruption_estimate,access_to_electricity_pct,...,inflation_yr_pct,tax_revenue_pct,gov_effectiveness_estimate,gdp_current_usd_M,political_stability_estimate,rule_of_law_estimate,gov_exp_on_education_pct,gov_health_exp_pct,life_expectancy_at_birth,voice_and_accountability_estimate
0,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1960,58.19,1.85,-1.44,45.75,...,4.98,7.65,-1.48,537.78,-2.51,-1.71,3.36,0.53,32.53,-1.19
1,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1961,57.88,1.85,-1.44,45.75,...,4.98,7.65,-1.48,548.89,-2.51,-1.71,3.36,0.53,33.07,-1.19
2,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1962,57.96,1.85,-1.44,45.75,...,4.98,7.65,-1.48,546.67,-2.51,-1.71,3.36,0.53,33.55,-1.19
3,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1963,58.03,1.85,-1.44,45.75,...,4.98,7.65,-1.48,751.11,-2.51,-1.71,3.36,0.53,34.02,-1.19
4,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1964,58.12,1.85,-1.44,45.75,...,4.98,7.65,-1.48,800.00,-2.51,-1.71,3.36,0.53,34.49,-1.19


# Basic Inspection

In [3669]:
# Check for fully identical rows across all columns
fully_duplicate_rows = cleanmerged_df[cleanmerged_df.duplicated(keep=False)]

# Show the fully identical rows
print(fully_duplicate_rows)


Empty DataFrame
Columns: [country, code, region, income_group, lending_category, year, agricultural_land_pct, forest_land_pct, control_of_corruption_estimate, access_to_electricity_pct, renewable_energy_consumption_pct, co2_emissions, pop_density, inflation_yr_pct, tax_revenue_pct, gov_effectiveness_estimate, gdp_current_usd_M, political_stability_estimate, rule_of_law_estimate, gov_exp_on_education_pct, gov_health_exp_pct, life_expectancy_at_birth, voice_and_accountability_estimate]
Index: []

[0 rows x 23 columns]


In [3670]:
# 1. Display the first, last, and a random sample of 7 entries
print("First 5 rows:")
display(cleanmerged_df.head())

print("\nLast 5 rows:")
display(cleanmerged_df.tail())

print("\nRandom sample of 7 rows:")
display(cleanmerged_df.sample(n=7, random_state=42))  # set random_state for reproducibility

# 2. Check data types of all columns
print("\nData types of each column:")
display(cleanmerged_df.dtypes)

First 5 rows:


,country,code,region,income_group,lending_category,year,agricultural_land_pct,forest_land_pct,control_of_corruption_estimate,access_to_electricity_pct,...,inflation_yr_pct,tax_revenue_pct,gov_effectiveness_estimate,gdp_current_usd_M,political_stability_estimate,rule_of_law_estimate,gov_exp_on_education_pct,gov_health_exp_pct,life_expectancy_at_birth,voice_and_accountability_estimate
0,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1960,58.19,1.85,-1.44,45.75,...,4.98,7.65,-1.48,537.78,-2.51,-1.71,3.36,0.53,32.53,-1.19
1,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1961,57.88,1.85,-1.44,45.75,...,4.98,7.65,-1.48,548.89,-2.51,-1.71,3.36,0.53,33.07,-1.19
2,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1962,57.96,1.85,-1.44,45.75,...,4.98,7.65,-1.48,546.67,-2.51,-1.71,3.36,0.53,33.55,-1.19
3,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1963,58.03,1.85,-1.44,45.75,...,4.98,7.65,-1.48,751.11,-2.51,-1.71,3.36,0.53,34.02,-1.19
4,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA,1964,58.12,1.85,-1.44,45.75,...,4.98,7.65,-1.48,800.00,-2.51,-1.71,3.36,0.53,34.49,-1.19



Last 5 rows:


,country,code,region,income_group,lending_category,year,agricultural_land_pct,forest_land_pct,control_of_corruption_estimate,access_to_electricity_pct,...,inflation_yr_pct,tax_revenue_pct,gov_effectiveness_estimate,gdp_current_usd_M,political_stability_estimate,rule_of_law_estimate,gov_exp_on_education_pct,gov_health_exp_pct,life_expectancy_at_birth,voice_and_accountability_estimate
13883,Zimbabwe,ZWE,Sub-Saharan Africa,Lower middle income,Blend,2019,41.88,45.21,-1.27,46.68,...,255.30,16.65,-1.32,21832.23,-0.94,-1.30,6.00,0.52,61.29,-1.16
13884,Zimbabwe,ZWE,Sub-Saharan Africa,Lower middle income,Blend,2020,41.88,45.09,-1.29,52.75,...,557.20,16.65,-1.36,21509.70,-1.05,-1.33,6.00,0.76,61.12,-1.11
13885,Zimbabwe,ZWE,Sub-Saharan Africa,Lower middle income,Blend,2021,41.88,44.97,-1.25,48.98,...,98.55,16.65,-1.30,28371.24,-0.95,-1.28,6.00,1.60,59.25,-1.14
13886,Zimbabwe,ZWE,Sub-Saharan Africa,Lower middle income,Blend,2022,34.07,46.82,-1.26,36.51,...,104.71,16.65,-1.25,27366.63,-0.88,-1.24,6.00,1.60,56.36,-1.10
13887,Zimbabwe,ZWE,Sub-Saharan Africa,Lower middle income,Blend,2023,34.07,46.82,-1.29,36.51,...,3.47,16.65,-1.29,6563.81,-0.94,-1.51,6.00,1.60,56.36,-1.31



Random sample of 7 rows:


,country,code,region,income_group,lending_category,year,agricultural_land_pct,forest_land_pct,control_of_corruption_estimate,access_to_electricity_pct,...,inflation_yr_pct,tax_revenue_pct,gov_effectiveness_estimate,gdp_current_usd_M,political_stability_estimate,rule_of_law_estimate,gov_exp_on_education_pct,gov_health_exp_pct,life_expectancy_at_birth,voice_and_accountability_estimate
2742,Colombia,COL,Latin America & Caribbean,Upper middle income,IBRD,2014,40.38,54.32,-0.40,97.79,...,2.90,15.70,-0.14,381240.86,-1.11,-0.25,4.63,5.06,76.04,-0.04
10644,Saudi Arabia,SAU,"Middle East, North Africa, Afghanistan & Pakistan",High income,Not classified,1980,40.45,0.45,-0.02,100.00,...,4.17,3.26,-0.07,164539.58,-0.47,0.10,5.94,2.88,62.70,-1.70
12248,Tanzania,TZA,Sub-Saharan Africa,Lower middle income,IDA,1984,35.00,58.28,-0.55,12.65,...,36.15,11.04,-0.62,18395.38,-0.40,-0.42,3.58,1.49,51.55,-0.32
3052,Cote d'Ivoire,CIV,Sub-Saharan Africa,Lower middle income,Blend,2004,62.89,14.60,-1.24,52.45,...,1.46,10.01,-1.28,23510.58,-2.11,-1.42,2.91,0.56,51.36,-1.22
2623,Chile,CHL,Latin America & Caribbean,High income,IBRD,2023,20.64,21.94,1.32,98.14,...,8.47,17.46,1.07,37846.19,0.43,1.19,3.76,3.28,73.07,1.02
13383,Vanuatu,VUT,East Asia & Pacific,Lower middle income,IDA,1967,9.27,36.28,0.23,32.50,...,3.05,17.49,-0.41,270.01,1.07,0.40,5.78,2.22,53.44,0.59
4356,France,FRA,Europe & Central Asia,High income,Not classified,1964,62.29,29.06,1.31,100.00,...,3.21,20.54,1.44,94007.85,0.42,1.41,5.33,7.91,70.66,1.19



Data types of each column:


country                               object
code                                  object
region                                object
income_group                          object
lending_category                      object
year                                   int32
agricultural_land_pct                float64
forest_land_pct                      float64
control_of_corruption_estimate       float64
access_to_electricity_pct            float64
renewable_energy_consumption_pct     float64
co2_emissions                        float64
pop_density                          float64
inflation_yr_pct                     float64
tax_revenue_pct                      float64
gov_effectiveness_estimate           float64
gdp_current_usd_M                    float64
political_stability_estimate         float64
rule_of_law_estimate                 float64
gov_exp_on_education_pct             float64
gov_health_exp_pct                   float64
life_expectancy_at_birth             float64
voice_and_

In [3671]:
cleanmerged_df.shape

(13056, 23)

In [3672]:
from IPython.display import display, Markdown
# final cleaned dataframe
df = cleanmerged_df

# Short, concise descriptions for all columns (updated to cleaned column names)
column_descriptions = {
    "country": "Country",
    "code": "Country code",
    "region": "Geographic region",
    "income_group": "Income classification",
    "lending_category": "Lending group class",
    "year": "Year of measurement",
    "agricultural_land_pct": "Agri land % of total area",
    "forest_land_pct": "Forest % of total area",
    "control_of_corruption_estimate": "Estimate of corruption control",
    "access_to_electricity_pct": "Population with electricity access",
    "renewable_energy_consumption_pct": "Renewable energy % of total consumption",
    "co2_emissions": "CO2 emissions in kt",
    "pop_density": "Population per km²",
    "inflation_yr_pct": "Annual inflation rate",
    "tax_revenue_pct": "Tax revenue % of GDP",
    "gov_effectiveness_estimate": "Estimate of government effectiveness",
    "gdp_current_usd_M": "GDP in current USD (M)",
    "political_stability_estimate": "Estimate of political stability",
    "rule_of_law_estimate": "Estimate of rule of law",
    "gov_exp_on_education_pct": "Gov. education expenditure % of GDP",
    "gov_health_exp_pct": "Gov. health expenditure % of GDP",
    "life_expectancy_at_birth": "Life expectancy in years",
    "voice_and_accountability_estimate": "Estimate of voice and accountability"
}

# Function to format example values nicely
def format_example_values(series):
    # Choose 2 random unique values
    values = np.random.choice(series.dropna().unique(), size=2, replace=False) if series.nunique() >= 2 else series.dropna().unique()
    
    # Format floats with commas and 2 decimals
    formatted = [f"{v:,.2f}" if isinstance(v, (float, np.floating)) else v for v in values]
    return formatted

# Build the dataset structure table
structure_table = pd.DataFrame({
    "Feature/variable": df.columns,
    "Data type": [df[col].dtype for col in df.columns],
    "Description": [column_descriptions.get(col, "") for col in df.columns],
    "# Unique values": [df[col].nunique() for col in df.columns],
    "Eg. values": [format_example_values(df[col]) for col in df.columns]
})

# Display the table
pd.set_option("display.max_rows", None)  # show all rows
structure_table

# Generate Markdown table string
markdown_table = structure_table.to_markdown(index=False)

# Print it so you can copy-paste into your report
print(markdown_table)


| Feature/variable                  | Data type   | Description                             |   # Unique values | Eg. values                                 |
|:----------------------------------|:------------|:----------------------------------------|------------------:|:-------------------------------------------|
| country                           | object      | Country                                 |               204 | ['Bahrain', 'Vietnam']                     |
| code                              | object      | Country code                            |               204 | ['ISR', 'FIN']                             |
| region                            | object      | Geographic region                       |                 7 | ['Europe & Central Asia', 'North America'] |
| income_group                      | object      | Income classification                   |                 4 | ['High income', 'Low income']              |
| lending_category                  | object  

In [3673]:
# Select one numeric column per category 
selected_cols = [
    'gdp_current_usd_M',          # Economic
    'co2_emissions',              # Environmental
    'gov_effectiveness_estimate', # Governance
    'life_expectancy_at_birth'    # Demographic
]

# Compute descriptive statistics
desc = cleanmerged_df[selected_cols].describe().round(2)

# Convert to markdown
markdown_table = desc.to_markdown()
print(markdown_table)

|       |   gdp_current_usd_M |    co2_emissions |   gov_effectiveness_estimate |   life_expectancy_at_birth |
|:------|--------------------:|-----------------:|-----------------------------:|---------------------------:|
| count |     13056           |  13056           |                     13056    |                   13056    |
| mean  |    166064           | 143573           |                        -0.02 |                      64.63 |
| std   |    934961           | 622493           |                         0.99 |                      11.29 |
| min   |         8.82        |      0           |                        -2.44 |                      12    |
| 25%   |      1534.1         |   1747.98        |                        -0.75 |                      57.45 |
| 50%   |      7745.25        |   9563.14        |                        -0.16 |                      67.54 |
| 75%   |     46537           |  55132.7         |                         0.69 |                      73.06 |
|

In [3674]:
#Category Columns
category_columns = cleanmerged_df.select_dtypes(include=['object']).columns
category_columns

Index(['country', 'code', 'region', 'income_group', 'lending_category'], dtype='object')

In [3675]:
# List of category columns excluding 'code'
category_columns = [col for col in ['country', 'region', 'income_group', 'lending_category'] if col in cleanmerged_df.columns]

# Build category summary
cat_summary = {}

for col in category_columns:
    vc = cleanmerged_df[col].value_counts(dropna=False)
    cat_summary[col] = [
        cleanmerged_df[col].count(),
        cleanmerged_df[col].nunique(dropna=False),
        vc.idxmax(),
        vc.max(),
        vc.idxmin(),
        vc.min()
    ]

# Create DataFrame
cat_summary_table = pd.DataFrame(
    cat_summary,
    index=[
        'Count',
        'Number of unique values',
        'Most frequent value',
        'Most frequent value (frequency)',
        'Least frequent value',
        'Least frequent value (frequency)'
    ]
)

# Display as markdown
print(cat_summary_table.to_markdown())


|                                  | country     | region                | income_group   | lending_category   |
|:---------------------------------|:------------|:----------------------|:---------------|:-------------------|
| Count                            | 13056       | 13056                 | 13056          | 13056              |
| Number of unique values          | 204         | 7                     | 4              | 4                  |
| Most frequent value              | Afghanistan | Europe & Central Asia | High income    | IBRD               |
| Most frequent value (frequency)  | 64          | 3392                  | 4736           | 4416               |
| Least frequent value             | Afghanistan | North America         | Low income     | Blend              |
| Least frequent value (frequency) | 64          | 192                   | 1664           | 1216               |



## List of Assumptions:

1. Missing Values
    Region & Income group:
       - If missing, imputed using the most frequent value in the country’s region.
       - For unique cases (like Venezuela), the income group was assigned based on most frequent 
         income in its region.

    Country-specific imputation:
    - For Korea, missing indicators were filled using median values of countries in the same region 
      and closest income group (Lower middle income).
    - Ethiopia’s missing income group was imputed based on its region (Sub-SaharanAfrica) and comparison 
      with similar countries.
    - Countries with no data in 8+ indicators were dropped eg. Gibraltar, St. Martin (French part), 
      French Polynesia

    Lending category:
    - High-income countries with missing lending category values were set as "Not classified".
    - Other countries’ lending category was imputed using the most frequent lending category in the same 
      Region and Income group.

2. Impossible Values

    - Checked that numeric columns cannot have negative values where it doesn’t make sense, e.g., 
      population_density, GDP_current_US, CO2_emissions, tax_revenue%, government_expenditure_on_education%,
      government_health_expenditure%, life_expectancy_at_birth.
    - inflation_annual% had negative values but we assumed that is deflation.

3. Column Selection / Dropping

    - Dropped columns based on:
        Redundancy: Standard deviation columns for governance indicators, duplicate variables.

    - High missingness: 
        Columns with many missing values (e.g., human_capital_index, research_and_development_expenditure%).

    - Irrelevance / complexity: 
        Indicators not directly relevant or technically complex for analysis (e.g., military expenditure, 
        rural population, birth/death rate).

Our assumption is that the remaining columns are sufficient for governance, economic, environmental, and demographic analysis.

4. Data Types

    - Ensured all columns have appropriate data types:
        Numeric for indicators, categorical for Region, Income group, Lending category, etc.


## Categorizing Indicators
For analysis purposes, we decided to keep the following indicators in the assigned categories.

### Governance and institutional quality indicators:
 1. goverment_effectiveness_estimate
 2. control_of_corruption_estimate
 3. rule_of_law_estimate
 4. voice_and_accountability_estimate
 5. political_stability_estimate

### Environmental sustainability indicators:
 1. CO2_emisions
 2. renewvable_energy_consumption%
 3. forest_land%
 4. agricultural_land

### Economic performance and poverty reduction indicators
 1. GDP_current_US
 2. inflation_annual%
 3. tax_revenue%

### Human well-being and health indicators:
 1. life_expectancy_at_birth
 2. government_expenditure_on_education%
 3. government_health_expenditure%
 4. access_to_electricity%
 5. population_density